# Pointers & dynamic memory management

Scott Meyers: *A resource is something that, once you’re done using it, you need to return to the system. If you don’t, bad things happen. In C++ programs, the most commonly used resource is dynamically allocated memory (if you allocate memory and never deallocate it, you’ve got a memory leak), but memory is only one of many resources you must manage. Other common resources include file descriptors, mutex locks, fonts and brushes in graphical user interfaces (GUIs), database connections, and network sockets. Regardless of the resource, it’s important that it be released when you’re finished with it. Trying to ensure this by hand is difficult under any conditions, but when you consider exceptions, functions with multiple return paths, and maintenance programmers modifying software without fully comprehending the impact of their changes, it becomes clear that ad hoc ways of dealing with resource management aren’t sufficient.*

## Motivation

Have a look at this toy library for particles.

In [ ]:
%%file tmp.particles.h

class Particle {
  public :
    virtual ~Particle() {}
    virtual char const * name() const =0 ;
} ;

class Electron : public Particle {
  public :
    virtual char const * name() const
     { return "Electron" ; }
} ;

class Proton : public Particle {
  public :
    virtual char const * name() const
     { return "Proton" ; }
} ;
 
class Neutron : public Particle {
  public :
    virtual char const * name() const
     { return "Neutron" ; }
} ;

Particle * new_particle() {
  int const NB = 3 ;
  int num = NB*(std::rand()/(double(RAND_MAX)+1)) ;
  switch (num)
   {
    case 0: return new Electron() ;
    case 1: return new Proton() ;
    case 2: return new Neutron() ;
   }
  return 0 ;
}

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include <iostream>
#include <ctime>
    
void print( Particle const * p ) {
  std::cout<<p->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    Particle * p = new_particle() ;
    print(p) ;
    delete p ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

In real-life long functions, the `delete` can be bypassed, due to:
* a premature call to `return`, `break`, `continue`... ;
* an exception is raised.

## Solution : RAII idiom

In order to make sure that a *resource* is released at the end of its utilization, one can wrap this resource into a *guard* object, whose **destructor** will release the resource. There is no more possible bypass, because whenever one leave a function, all local objects are **always destructed**.

In [ ]:
%%file tmp.guards.h

class ParticleGuard {
  public :
    explicit ParticleGuard( Particle * p ) : m_p(p) {}
    ~ParticleGuard() { delete m_p ; }
  private :
    Particle * m_p ;
} ;

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include "tmp.guards.h"
#include <iostream>
#include <ctime>
    
void print( Particle const * p ) {
  std::cout<<p->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    Particle * p = new_particle() ;
    ParticleGuard pg(p) ;
    print(p) ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

BEWARE: one should not create multiple guards for a single resource, or this resource will be released several times...

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include "tmp.guards.h"
#include <iostream>
#include <ctime>
    
void print( Particle const * p ) {
  std::cout<<p->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    Particle * p = new_particle() ;
    ParticleGuard pg1(p) ;
    ParticleGuard pg2(p) ;
    print(p) ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

## Access to the resource

To avoid this type of errors, the resource should be directly given to the guard... but we cannot access it any more ! Hence the need for some way to retreive the resource from the guard:

In [ ]:
%%file tmp.guards.h

class ParticleGuard {
  public :
    explicit ParticleGuard( Particle * p ) : m_p(p) {}
    ~ParticleGuard() { delete m_p ; }
    Particle * get() { return m_p ; }
  private :
    Particle * m_p ;
} ;

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include "tmp.guards.h"
#include <iostream>
#include <ctime>
    
void print( Particle const * p ) {
  std::cout<<p->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    ParticleGuard pg(new_particle()) ;
    print(pg.get()) ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

One may be tempted to improve the guard with some conversion operator.

In [ ]:
%%file tmp.guards.h
    
class ParticleGuard {
  public :
    explicit ParticleGuard( Particle * p ) : m_p(p) {}
    ~ParticleGuard() { delete m_p ; }
    operator Particle *() { return m_p ; }
  private :
    Particle * m_p ;
} ;

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include "tmp.guards.h"
#include <iostream>
#include <ctime>
    
void print( Particle const * p ) {
  std::cout<<p->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    ParticleGuard p(new_particle()) ;
    print(p) ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

**BEWARE**: in real-life long functions, this opens the door to tricky errors, when a guard is implicitly and invisibly transformed into its resource. That's why many library designers prefer some explicit `get()` function.

## How to safely copy the guards ?
 
What happens if we copy a guard ? This code will crash:

In [ ]:
%%file tmp.guards.h
    
class ParticleGuard {
  public :
    explicit ParticleGuard( Particle * p ) : m_p(p) {}
    ~ParticleGuard() { delete m_p ; }
    Particle * get() { return m_p ; }
  private :
    Particle * m_p ;
} ;

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include "tmp.guards.h"
#include <iostream>
#include <ctime>
    
void print( ParticleGuard pg ){
  std::cout<<pg.get()->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    ParticleGuard pg(new_particle()) ;
    print(pg) ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

## Minimal strategy: forbid the copy

<img src="img/guards-nocopy.png">

In [ ]:
%%file tmp.guards.h
    
class ParticleGuard {
  public :
    explicit ParticleGuard( Particle * p ) : m_p(p) {}
    ~ParticleGuard() { delete m_p ; }
    Particle * get() { return m_p ; }
  private :
    ParticleGuard( const ParticleGuard & ) ;
    ParticleGuard & operator=( const ParticleGuard & ) ;
    Particle * m_p ;
} ;

Our modified `print()` is not any more allowed:

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include "tmp.guards.h"
#include <iostream>
#include <ctime>
    
void print( ParticleGuard pg ){
  std::cout<<pg.get()->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    ParticleGuard pg(new_particle()) ;
    print(pg) ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In our example, we can easily get the expected result with some reference instead:

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include "tmp.guards.h"
#include <iostream>
#include <ctime>
    
void print( ParticleGuard & pg ){
  std::cout<<pg.get()->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    ParticleGuard pg(new_particle()) ;
    print(pg) ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

## Robust (but slow) strategy: internally count the guards

<img src="img/guards-sharedcopy.png"/>

All the guards must share a common control block.

In [ ]:
%%file tmp.guards.h
    
class ParticleControlBlock {
  public:
    unsigned int m_nb_guards ;
    explicit ParticleControlBlock( Particle * p )
     : m_p(p), m_nb_guards(1) {}
    ~ParticleControlBlock()
     { delete m_p ; }
    Particle * get() { return m_p ; }
  private:
    ParticleControlBlock( const ParticleControlBlock & ) ;
    ParticleControlBlock & operator=( const ParticleControlBlock & ) ;
    Particle * m_p ;
} ;

void increment_guards( ParticleControlBlock * block ) {
  block->m_nb_guards++ ;
}

void decrement_guards( ParticleControlBlock * block ) {
  block->m_nb_guards-- ;
  if (block->m_nb_guards==0)
   { delete block ; }
}

class ParticleGuard {
  public:
    explicit ParticleGuard( Particle * p )
     { m_block = new ParticleControlBlock(p) ; }                         
    ParticleGuard( const ParticleGuard & pg )
     {
      m_block = pg.m_block ;
      increment_guards(m_block) ; 
     }                        
    ParticleGuard & operator=( ParticleGuard const & pg )
     {
      if (this==&pg) return *this ;
      decrement_guards(m_block) ;
      m_block = pg.m_block ;
      increment_guards(m_block) ; 
      return *this ;
     }                         
    ~ParticleGuard()
     { decrement_guards(m_block) ; }
    Particle * get()
     { return m_block->get() ; }
  private :
    ParticleControlBlock * m_block ; 
} ;

In [ ]:
%%file tmp.raii.cpp

#include "tmp.particles.h"
#include "tmp.guards.h"
#include <iostream>
#include <ctime>
    
void print( ParticleGuard pg ){
  std::cout<<pg.get()->name()<<std::endl ;
}

int main() {
  std::srand(std::time(0)) ;
  for ( int i = 0 ; i < 5 ; ++i ) {
    ParticleGuard pg(new_particle()) ;
    print(pg) ;
  }
}

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

## Simple (but not universal) strategy : deep copy

Sometimes it is preferable to duplicate the resource each time its guard is copied.

<img src="img/guards-deepcopy.png"/>

In our example, there is the additional difficulty that the real class of the pointed particle is not known, hence the need for some virtual `clone` function in the `Particle` inheritance tree.

In [ ]:
%%file tmp.particles.h

#include <cstdlib> // rand()

class Particle {
  public :
    virtual ~Particle() {}
    virtual const char * name() const =0 ;
    virtual Particle * clone() const =0 ;
} ;

class Electron : public Particle {
  public :
    virtual const char * name() const { return "Electron" ; }
    virtual Particle * clone() const { return new Electron(*this) ; }
} ;

class Proton : public Particle {
  public :
    virtual const char * name() const { return "Proton" ; }
    virtual Particle * clone() const { return new Proton(*this) ; }
} ;
 
class Neutron : public Particle {
  public :
    virtual const char * name() const { return "Neutron" ; }
    virtual Particle * clone() const { return new Neutron(*this) ; }
} ;

Particle * new_particle() {
  const int NB = 3 ;
  int num = NB*(std::rand()/(double(RAND_MAX)+1)) ;
  switch (num)
   {
    case 0: return new Electron() ;
    case 1: return new Proton() ;
    case 2: return new Neutron() ;
   }
  return 0 ;
}

In [ ]:
%%file tmp.guards.h
    
class ParticleGuard {
  public:
    explicit ParticleGuard( Particle * p ) : m_p(p) {}
    ParticleGuard( const ParticleGuard & pg )
     { m_p = pg.m_p->clone() ; }                       
    ParticleGuard & operator=( const ParticleGuard & pg )
     {
      if (this==&pg) return (*this) ; 
      delete m_p ; m_p = pg.m_p->clone() ;
      return (*this) ;
     }
    ~ParticleGuard() { delete m_p ; }
    Particle * get() { return m_p ; }
  private :
    Particle * m_p ; 
} ;

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

**NOTE**: `std::string` is a kind of front-end for a dynamically allocated array of characters. That's why it has no maximum size. When a string is duplicated, there is a deep copy.

## Subtle (but unnatural) strategy : transfer the resource

In rare cases, we want to allow the copy of a guard, stealing the resource from the former guard, and letting it *empty*.

<img src="img/guards-move.png"/>

In [ ]:
%%file tmp.guards.h
    
class ParticleGuard {
  public:
    explicit ParticleGuard( Particle * p ) : m_p(p) {}
    ParticleGuard( ParticleGuard & pg )
     { m_p = pg.m_p ; pg.m_p = 0 ; }                       
    ParticleGuard & operator=( ParticleGuard & pg )
     {
      if (this==&pg) return (*this) ; 
      delete m_p ; m_p = pg.m_p ; pg.m_p = 0 ;
      return (*this) ;
     }
    ~ParticleGuard() { delete m_p ; }
    Particle * get() { return m_p ; }
  private :
    Particle * m_p ; 
} ;

In [ ]:
!rm -f tmp.raii.exe && g++ -std=c++17 tmp.raii.cpp -o tmp.raii.exe

In [ ]:
!./tmp.raii.exe

**BEWARE**: this requires an unusual copy constructor, which modify the original guard. The infamous class `std::auto_ptr` was implemented this way, with the major limitation that one cannot store some `std::auto_ptr` in some `std::vector`. Solving this issue requires a new feature : the ***move semantic*** .

# Take away

- Pointers stay both the blessing and the malediction of the language.
- Modern C++ will make them a lot less necessary or a lot more hidden in the standard library.
- A key elegant feature is the new **rvalue reference** and the associated **move semantic**.

# Questions ?

© *CNRS 2024*  
*This document was created by David Chamont and translated by Olga Abramkina. It is available under the [License Creative Commons - Attribution - No commercial use - Shared under the conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/)*